# Build Tech tree representation
Natalia Vélez, June 2020

In [1]:
%matplotlib inline
import os, re, glob, json, operator
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from os.path import join as opj
from tqdm import notebook

import ohol_transitions as transition
import ohol_object as obj

## Setup

### Find data

Find all transition files:

In [2]:
gsearch = lambda *args: glob.glob(opj(*args))

data_dir = '../../OneLifeData7/'
trans_dir  = opj(data_dir, 'transitions')
trans_files = gsearch(trans_dir, '*.txt')

print(*trans_files[:5], sep='\n')

../../OneLifeData7/transitions/-1_2574.txt
../../OneLifeData7/transitions/0_702.txt
../../OneLifeData7/transitions/314_235.txt
../../OneLifeData7/transitions/2165_2165.txt
../../OneLifeData7/transitions/0_1692.txt


Find all object files:

In [3]:
obj_dir = opj(data_dir, 'objects')
obj_files = gsearch(obj_dir, '*.txt')

print(*obj_files[:5], sep='\n')

../../OneLifeData7/objects/3644.txt
../../OneLifeData7/objects/1053.txt
../../OneLifeData7/objects/1735.txt
../../OneLifeData7/objects/3122.txt
../../OneLifeData7/objects/2228.txt


### Data check
Do all non-special transition files have associated object files?

a. Object files:

In [4]:
all_obj_objs = []
for f in obj_files:
    f_base = os.path.basename(f)
    fname, _ =  os.path.splitext(f_base)
    all_obj_objs.append(fname)

all_obj_objs.sort()

is_special_obj = re.compile('[a-z]+')
print(*[o for o in all_obj_objs if is_special_obj.match(o)], sep='\n')

groundHeat_4
groundHeat_5
groundHeat_6
nextObjectNumber


Filter out specials:

In [5]:
obj_objs = np.array([int(o) for o in all_obj_objs if not is_special_obj.match(o)])
print(*obj_objs[:5], sep='\n')

100
1000
1001
1002
1003


b. Transition files:

In [6]:
trans_objs = []
for f in trans_files:
    f_objs = transition.read_objs(f)
    trans_objs.append(f_objs)

In [7]:
all_trans_objs = np.array(trans_objs)
all_trans_objs = all_trans_objs.flatten()
#all_trans_objs = all_trans_objs[all_trans_objs > 0]
unique_trans_objs = np.unique(all_trans_objs)
unique_trans_objs

array([  -2,   -1,    0, ..., 4263, 4264, 4265])

Any transition objs unaccounted for?:

In [8]:
np.setdiff1d(unique_trans_objs, obj_objs)

array([-2, -1,  0])

^ -2, 1, 0 are all special designations for empty spaces and player characters. We are ready to go!

## Build graph from transitions

First, we'll make note of each object's ingredients and products, across all transitions:

In [9]:
transition_dict = {}

for f in notebook.tqdm(trans_files):
    tdata = transition.read_transition(f)
    t_from = (tdata['origActor'], tdata['origTarget'])
    t_to = (tdata['newActor'], tdata['newTarget'])
    
    for o in t_to:
        if (o > 0) & (o not in t_from):
        
            if o not in transition_dict:
                transition_dict[o] = {'ingredients': [], 'products': [], 'recipes': []}
            
            transition_dict[o]['ingredients'].append(t_from)            
            
    
    for o in t_from:
        
        if (o > 0) & (o not in t_to):
            if o not in transition_dict:
                    transition_dict[o] = {'ingredients': [], 'products': [], 'recipes': []}
        
            transition_dict[o]['products'].append(t_to) 

Clean up ingredients and products:

In [10]:
for k,v in transition_dict.items():
    for i_k, i_v in transition_dict[k].items():
        
        i_v_new = np.array(i_v).flatten()
        i_v_new = i_v_new[i_v_new > 0].tolist()
        transition_dict[k][i_k] = i_v_new

Next, we'll use the ingredients + products list to identify "legal" transitions. 

**Illegal transitions**

We want to avoid endless loops, where products are reverted back to their components. To do this effectively, we do have to retrace the tree back a few steps, otherwise we will miss a few transformations:

Example 1:

* carrot + bowl —> bowl of carrot (legal!)
* empty hand + bowl of carrot —> carrot + bowl (illegal!)

Example 2: 

* clay bowl + rabbit —> bowl of rabbit + empty floor
* sharp stone + rabbit —> sharp stone + bowl of minced rabbit
* empty hand + bowl of minced rabbit —> minced rabbit + **clay bowl** (illegal!)

We also want to avoid tools being listed as their own components. For example, take a closer look at this transition:

**sharp stone** + rabbit —> **sharp stone** + bowl of minced rabbit

We want sharp stone + rabbit to be added to the recipes for "bowl of minced rabbit", but not for "sharp stone."

`get_precursors` is a helper function to check that condition A is met

In [17]:
def get_precursors(product, depth):
    
    precursors = [i for i in transition_dict[product]['ingredients'] if i > 0]
    
    if depth > 0:
        for p in precursors:
            if p in transition_dict:
                precursors += get_precursors(p, depth-1)
        
    return precursors

def is_noncirc(i_from, i_to): return i_to not in transition_dict[i_from]['ingredients']

Iterate over dictionary:

In [19]:
for f in notebook.tqdm(trans_files):
    tdata = transition.read_transition(f)
    t_from = (tdata['origActor'], tdata['origTarget'])
    t_to = (tdata['newActor'], tdata['newTarget'])
    
    
    for product in t_to:
        if (product > 0) & (product not in t_from):
            
            prec = get_precursors(product, 0)
            
            is_valid = np.all([is_noncirc(ingredient, product) for ingredient in prec if ingredient > 0])
            
            if is_valid:
                transition_dict[product]['recipes'].append(t_from)

KeyError: 519

In [40]:
for k,v in transition_dict.items():
    
    if not transition_dict[k]['ingredients']:
        print(obj.read_obj(k)['name'])
    

Schnauser with Puppies -3
Collie with Puppies -3
Fishing Pole with Char - just caught
@ Non-tilling Skewer
Green Bowler Hat with Feather - justDyed
@ Carrot
Holey Net
Wild Cucumber Plant
Yellow Top Hat with Feather - justDyed
Soda Ash Bottle - +contLabDish
Stack of Floppy Baskets
@ Any Card
Deep Well - was empty +famUse100
Sugarcane Stalk
Indigo
@ Small Trash
Red Bowler Hat with Feather - justDyed
@ Wet Canada Goose Pond
East West Track
Proposed Property Fence - groundOnly +cornerA
Pit Bull with Puppies -3
Chihuahua with Puppies -3
Dachshund with Puppies -3
Rose Madder
Wild Cabbage
Dry Planted Onions
Indigo Top Hat with Feather - justDyed
@ Yew Bow
@ Any fence that renews corner
White Rose
Skim Milk Bottle - +contFoodDish +drink
Baby Bone Pile - origGrave justDied fromDeath
Cut Glasswort Pile
Shallow Well - was empty +famUse100
@ Tillable Row
@ Young Stone Wall
@ Deadly Wolf
Green Plaster Wall - +causeAutoOrient shelfRemoved
@ Southward Entering Tracks
Firing Newcomen Hammer - +toolThe

In [44]:
obj.read_obj(2902)

{'id': 2902,
 'name': '@ Dyed Indigo Item Pattern',
 'containable': 0,
 'containSize': 1.0,
 'vertSlotRot': -0.25,
 'permanent': 0,
 'minPickupAge': 3,
 'noFlip': 0,
 'sideAccess': 0,
 'heldInHand': 1,
 'blocksWalking': 0,
 'leftBlockingRadius': 0,
 'rightBlockingRadius': 0,
 'drawBehindPlayer': 0,
 'mapChance': 0.0,
 'heatValue': 0,
 'rValue': 0.0,
 'person': 0,
 'noSpawn': 0,
 'male': 0,
 'deathMarker': 0,
 'homeMarker': 0,
 'floor': 0,
 'floorHugging': 0,
 'foodValue': 0,
 'speedMult': 1.0,
 'heldOffset': [0.0, 0.0],
 'clothing': 'n',
 'clothingOffset': [0.0, 0.0],
 'deadlyDistance': 0,
 'useDistance': 1,
 'sounds': ['-1:0.0', '-1:0.0', '-1:0.0', '-1:0.0'],
 'creationSoundInitialOnly': 0,
 'creationSoundForce': 0,
 'numSlots': 0,
 'timeStretch': 1.0,
 'slotSize': 1.0,
 'slotsLocked': 0,
 'numSprites': 1,
 'spriteID': 711,
 'pos': [0.0, 0.0],
 'rot': 0.0,
 'hFlip': 0,
 'color': [0.082353, 0.403922, 0.811765],
 'ageRange': [-1.0, -1.0],
 'parent': -1,
 'invisHolding': 0,
 'invisWorn':

In [24]:
transition.read_transition(opj(trans_dir, '185_519.txt'))

{'origActor': 185,
 'origTarget': 519,
 'newActor': 187,
 'newTarget': 519,
 'autoDecaySeconds': 0,
 'actorMinUseFraction': 0.0,
 'targetMinUseFraction': 0.0,
 'reverseUseActor': 0,
 'reverseUseTarget': 0,
 'move': 0,
 'desiredMoveDist': 1,
 'lastUseActor': False,
 'lastUseTarget': False,
 'origActorName': 'Skewered Rabbit',
 'origTargetName': '@ Food Burner',
 'newActorName': 'Burnt Rabbit',
 'newTargetName': '@ Food Burner',
 'isTool': False}

Now we will recursively go through each transition and add depth: